In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error
import torch.optim.lr_scheduler as lr_scheduler
import torch.nn.functional as F


In [3]:
data=pd.read_csv("Powersupply.csv")

In [4]:
data_trimmed = data.iloc[:1500]
data_trimmed.to_csv("Enhanced_India_Weather_Power_Supply.csv", index=False)


In [5]:
data.head(2)

,City,State,Area/Region,Date & Time,Weather Condition,Temperature (?C),Humidity (%),Rainfall Chances (%),Wind Speed (km/h),Fan Power Consumed (kWh),...,Grinder Power Consumed (kWh),AC Power Consumed (kWh),Heater Power Consumed (kWh),Fridge Power Consumed (kWh),TV Power Consumed (kWh),Total Power Consumed (kWh),Required Power Supply (kW),Current Power Supply (kW),Power Being Used (kW),Power Supply Status
0,Mumbai,Maharashtra,Bandra,01/01/2024 0:00,Rainy,25.3,66,43,43,0.35,...,7.17,7.25,7.10,4.11,2.02,42.13,423,301,301,Underloss
1,Kolkata,West Bengal,Garia,01/01/2024 0:01,Sunny,18.5,36,87,25,0.03,...,6.78,10.17,5.78,2.66,1.43,32.53,301,489,301,Excess


In [6]:
df = pd.DataFrame(data)
df = df.drop(columns=['Date & Time'])
#df=df.drop(columns=['Power Supply Status'])

In [7]:
df.head(10)

,City,State,Area/Region,Weather Condition,Temperature (?C),Humidity (%),Rainfall Chances (%),Wind Speed (km/h),Fan Power Consumed (kWh),Light Power Consumed (kWh),...,Grinder Power Consumed (kWh),AC Power Consumed (kWh),Heater Power Consumed (kWh),Fridge Power Consumed (kWh),TV Power Consumed (kWh),Total Power Consumed (kWh),Required Power Supply (kW),Current Power Supply (kW),Power Being Used (kW),Power Supply Status
0,Mumbai,Maharashtra,Bandra,Rainy,25.3,66,43,43,0.35,0.17,...,7.17,7.25,7.10,4.11,2.02,42.13,423,301,301,Underloss
1,Kolkata,West Bengal,Garia,Sunny,18.5,36,87,25,0.03,0.08,...,6.78,10.17,5.78,2.66,1.43,32.53,301,489,301,Excess
2,Ahmedabad,Gujarat,Maninagar,Rainy,17.2,64,19,48,0.04,0.23,...,6.52,5.83,33.54,0.15,1.30,52.33,260,326,260,Excess
3,Bengaluru,Karnataka,JP Nagar,Rainy,29.5,92,36,8,0.65,0.20,...,6.20,17.34,34.00,3.68,0.29,74.94,409,556,409,Excess
4,Kolkata,West Bengal,Salt Lake,Rainy,15.6,56,38,18,0.17,0.14,...,3.68,26.64,16.33,1.38,1.34,52.75,286,194,194,Underloss
5,Hyderabad,Telangana,Begumpet,Rainy,27.6,42,78,34,0.36,0.17,...,5.43,7.98,9.70,1.77,2.33,39.01,288,579,288,Excess
6,Mumbai,Maharashtra,Borivali,Clear,22.8,73,58,23,0.87,0.20,...,2.49,17.23,23.82,1.91,0.71,56.94,114,468,114,Excess
7,Jaipur,Rajasthan,Mansarovar,Clear,28.1,76,40,13,0.84,0.04,...,1.01,14.78,35.69,3.76,1.73,68.71,170,441,170,Excess
8,Bengaluru,Karnataka,JP Nagar,Cloudy,27.2,55,37,3,1.16,0.14,...,6.94,11.47,4.47,1.23,1.30,30.82,375,492,375,Excess
9,Chennai,Tamil Nadu,Anna Nagar,Sunny,17.6,44,31,25,0.53,0.20,...,3.12,29.00,23.34,0.23,1.29,66.88,395,140,140,Underloss


In [8]:
df.isna().sum()

City                                    0
State                                   0
Area/Region                             0
Weather Condition                       0
Temperature (?C)                        0
Humidity (%)                            0
Rainfall Chances (%)                    0
Wind Speed (km/h)                       0
Fan Power Consumed (kWh)                0
Light Power Consumed (kWh)              0
Mixer Power Consumed (kWh)              0
Washing Machine Power Consumed (kWh)    0
Phone Charging Power Consumed (kWh)     0
UPS Power Consumed (kWh)                0
Grinder Power Consumed (kWh)            0
AC Power Consumed (kWh)                 0
Heater Power Consumed (kWh)             0
Fridge Power Consumed (kWh)             0
TV Power Consumed (kWh)                 0
Total Power Consumed (kWh)              0
Required Power Supply (kW)              0
Current Power Supply (kW)               0
Power Being Used (kW)                   0
Power Supply Status               

In [9]:
cols_encoder=['City','State','Area/Region','Weather Condition','Power Supply Status']

In [10]:
label_encoders = {}
for col in cols_encoder:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [11]:
df.head(5)

,City,State,Area/Region,Weather Condition,Temperature (?C),Humidity (%),Rainfall Chances (%),Wind Speed (km/h),Fan Power Consumed (kWh),Light Power Consumed (kWh),...,Grinder Power Consumed (kWh),AC Power Consumed (kWh),Heater Power Consumed (kWh),Fridge Power Consumed (kWh),TV Power Consumed (kWh),Total Power Consumed (kWh),Required Power Supply (kW),Current Power Supply (kW),Power Being Used (kW),Power Supply Status
0,8,3,3,3,25.3,66,43,43,0.35,0.17,...,7.17,7.25,7.10,4.11,2.02,42.13,423,301,301,2
1,6,8,13,5,18.5,36,87,25,0.03,0.08,...,6.78,10.17,5.78,2.66,1.43,32.53,301,489,301,0
2,0,1,27,3,17.2,64,19,48,0.04,0.23,...,6.52,5.83,33.54,0.15,1.30,52.33,260,326,260,0
3,1,2,22,3,29.5,92,36,8,0.65,0.20,...,6.20,17.34,34.00,3.68,0.29,74.94,409,556,409,0
4,6,8,32,3,15.6,56,38,18,0.17,0.14,...,3.68,26.64,16.33,1.38,1.34,52.75,286,194,194,2


In [12]:
df.columns

Index(['City', 'State', 'Area/Region', 'Weather Condition', 'Temperature (?C)',
       'Humidity (%)', 'Rainfall Chances (%)', 'Wind Speed (km/h)',
       'Fan Power Consumed (kWh)', 'Light Power Consumed (kWh)',
       'Mixer Power Consumed (kWh)', 'Washing Machine Power Consumed (kWh)',
       'Phone Charging Power Consumed (kWh)', 'UPS Power Consumed (kWh)',
       'Grinder Power Consumed (kWh)', 'AC Power Consumed (kWh)',
       'Heater Power Consumed (kWh)', 'Fridge Power Consumed (kWh)',
       'TV Power Consumed (kWh)', 'Total Power Consumed (kWh)',
       'Required Power Supply (kW)', 'Current Power Supply (kW)',
       'Power Being Used (kW)', 'Power Supply Status'],
      dtype='object')

In [13]:
input_columns = ['City', 'State', 'Area/Region', 'Weather Condition', 'Temperature (?C)',
       'Humidity (%)', 'Rainfall Chances (%)', 'Wind Speed (km/h)',
       'Fan Power Consumed (kWh)', 'Light Power Consumed (kWh)',
       'Mixer Power Consumed (kWh)', 'Washing Machine Power Consumed (kWh)',
       'Phone Charging Power Consumed (kWh)', 'UPS Power Consumed (kWh)',
       'Grinder Power Consumed (kWh)', 'AC Power Consumed (kWh)',
       'Heater Power Consumed (kWh)', 'Fridge Power Consumed (kWh)',
       'TV Power Consumed (kWh)'
]
target_columns = [ 'Total Power Consumed (kWh)',
       'Required Power Supply (kW)', 'Current Power Supply (kW)',
       'Power Being Used (kW)']


In [14]:
X=df[input_columns].values
y=df[target_columns].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

In [16]:
X_train = scaler_X.fit_transform(X_train)
X_val = scaler_X.transform(X_val)
y_train = scaler_y.fit_transform(y_train)
y_val = scaler_y.transform(y_val)


In [17]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

In [18]:
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = torch.utils.data.TensorDataset(X_val_tensor, y_val_tensor)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

In [19]:
class AttentionLayer(nn.Module):
    def __init__(self, input_size):
        super(AttentionLayer, self).__init__()
        self.W = nn.Linear(input_size, input_size) 
        self.V = nn.Linear(input_size, 1)  

    def forward(self, x):
        scores = self.V(F.tanh(self.W(x))) 
        weights = F.softmax(scores, dim=1)  
        context = torch.bmm(weights.transpose(1, 2), x) 
        return context.squeeze(1) 

class PowerConsumptionModel(nn.Module):
    def __init__(self, input_size, output_size, dropout_rate=0.3):
        super(PowerConsumptionModel, self).__init__()
        self.attention1 = AttentionLayer(input_size)
        self.attention2 = AttentionLayer(input_size)
        
        self.network = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.BatchNorm1d(1024),
            nn.ELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(32, 16),
            nn.BatchNorm1d(16),
            nn.ELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(16, output_size),
        )

    def forward(self, x):
        context1 = self.attention1(x.unsqueeze(1))
        context2 = self.attention2(x.unsqueeze(1))
        combined_context = context1 + context2
        return self.network(combined_context)


In [20]:
input_size = X_train.shape[1]
output_size = y_train.shape[1]
model = PowerConsumptionModel(input_size, output_size)
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [44]:
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

epochs = 100
patience = 10
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        predictions = model(X_batch)
        loss = criterion(predictions, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    val_loss = 0
    val_mae = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            val_loss += loss.item()

            mae = torch.mean(torch.abs(predictions - y_batch))
            val_mae += mae.item()

    train_loss /= len(train_loader)
    val_loss /= len(val_loader)
    val_mae /= len(val_loader)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val MAE: {val_mae:.4f}")

    # Step the scheduler based on the validation loss
    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pth") 
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

model.load_state_dict(torch.load("best_model.pth"))
print("Best model loaded.")

C:\Users\dhany\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100, Train Loss: 0.3572, Val Loss: 0.3553, Val MAE: 0.6846
Epoch 2/100, Train Loss: 0.3561, Val Loss: 0.3556, Val MAE: 0.6866
Epoch 3/100, Train Loss: 0.3596, Val Loss: 0.3545, Val MAE: 0.6852
Epoch 4/100, Train Loss: 0.3633, Val Loss: 0.3547, Val MAE: 0.6871
Epoch 5/100, Train Loss: 0.3565, Val Loss: 0.3544, Val MAE: 0.6838
Epoch 6/100, Train Loss: 0.3579, Val Loss: 0.3544, Val MAE: 0.6836
Epoch 7/100, Train Loss: 0.3564, Val Loss: 0.3544, Val MAE: 0.6824
Epoch 8/100, Train Loss: 0.3586, Val Loss: 0.3544, Val MAE: 0.6832
Epoch 9/100, Train Loss: 0.3557, Val Loss: 0.3546, Val MAE: 0.6837
Epoch 10/100, Train Loss: 0.3567, Val Loss: 0.3546, Val MAE: 0.6840
Epoch 11/100, Train Loss: 0.3577, Val Loss: 0.3545, Val MAE: 0.6825
Epoch 12/100, Train Loss: 0.3589, Val Loss: 0.3540, Val MAE: 0.6817
Epoch 13/100, Train Loss: 0.3557, Val Loss: 0.3545, Val MAE: 0.6823
Epoch 14/100, Train Loss: 0.3555, Val Loss: 0.3550, Val MAE: 0.6838
Epoch 15/100, Train Loss: 0.3621, Val Loss: 0.3561, Val M

C:\Users\dhany\AppData\Local\Temp\ipykernel_16600\388780902.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


In [22]:
model.eval()

input_sample = X_val_tensor[10].unsqueeze(0) 
actual_target = y_val_tensor[10].unsqueeze(0)  

with torch.no_grad(): 
    output = model(input_sample)

predicted_output = scaler_y.inverse_transform(output.numpy())
actual_output = scaler_y.inverse_transform(actual_target.numpy())


print("Actual/Expected Output:", actual_output.round())
print("Predicted Output By Model:", predicted_output.round())

Actual/Expected Output: [[ 66. 393. 355. 355.]]
Predicted Output By Model: [[ 65. 293. 367. 233.]]
